In [1]:
#MAC USERS: 
#Before running ipynb - command line:
#ray stop
#ray start --head --port=6379 --redis-password="cbgt2"
#ray start --address='10.162.14.77:6379' --redis-password='cbgt2'
%load_ext autoreload
%autoreload 2
%reload_ext autoreload 
#if it doesn't work, shut down the ray server and start it again

In [2]:
#!autopep8 --in-place --aggressive --aggressive init_params.py
#!autopep8 --in-place --aggressive --aggressive popconstruct_copy.py
#!autopep8 --in-place --aggressive --aggressive generateepochs.py

In [3]:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct_copy as popconstruct
import generateepochs as gen
import pdb

2021-08-04 09:39:51,610	INFO worker.py:726 -- Connecting to existing Ray cluster at address: 192.168.0.235:6379


# Reward schedule

In [4]:
def GenRewardSchedule(n_trials, volatility, conflict, reward_mu, reward_std):
    
    reward_t1, reward_t2 = gen.define_reward(conflict, n_trials, reward_mu, reward_std)
    cp_idx, cp_indicator = gen.define_changepoints(n_trials, reward_t1, reward_t2, volatility)
    t1_epochs, t2_epochs, noisy_pattern, volatile_pattern = gen.define_epochs(n_trials, reward_t1, reward_t2, cp_idx, conflict)
    #print('t1:', reward_t1)
    #print('t2:', reward_t2)
    return volatile_pattern, cp_idx, cp_indicator, noisy_pattern, t1_epochs, t2_epochs

In [5]:
rsg = cbgt.Pipeline() # rsg is short for 'reward schedule generator'


(rsg.volatile_pattern,
 rsg.cp_idx,
 rsg.cp_indicator,
 rsg.noisy_pattern,
 rsg.t1_epochs,
 rsg.t2_epochs,
) = rsg[GenRewardSchedule](
        rsg.n_trials,
        rsg.volatility,
        rsg.conflict,
        rsg.reward_mu,
        rsg.reward_std,
    ).shape(6)

In [6]:
var_env = {
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}
results = cbgt.ExecutionManager(cores=7).run(rsg,var_env)
locals().update(results)

(pid=1163) t1: [0.85289709 0.         0.         1.01627861 0.99730769]
(pid=1163) t2: [0.         0.94675089 1.05500847 0.         0.        ]
(pid=1163) t1: [0.85289709 0.         0.         1.01627861 0.99730769]
(pid=1163) t2: [0.         0.94675089 1.05500847 0.         0.        ]


In [7]:
results.keys()

dict_keys(['n_trials', 'volatility', 'conflict', 'reward_mu', 'reward_std', 'volatile_pattern', 'cp_idx', 'cp_indicator', 'noisy_pattern', 't1_epochs', 't2_epochs'])

In [8]:
results['volatile_pattern']

array([0., 0., 1., 0., 1.])

In [9]:
results['cp_idx']

[0, 2, 3, 4]

In [10]:
results['cp_indicator']

array([1., 0., 1., 1., 1.])

In [11]:
results['noisy_pattern']

[1.0, 0.0, 1.0, 1.0, 0.0]

In [12]:
results['t1_epochs']

array([0.85289709, 0.        , 1.05500847, 1.01627861, 0.        ])

In [13]:
results['t2_epochs']

array([0.        , 0.94675089, 0.        , 0.        , 0.99730769])

# Net parameters 

In [14]:
pl = cbgt.Pipeline()

In [15]:
#DEFAULTS - init_params.py
pl.celldefaults = pl[par.helper_cellparams]()
pl.popspecific = pl[par.helper_popspecific]()
pl.receptordefaults = pl[par.helper_receptor]()
pl.basestim = pl[par.helper_basestim]()
pl.dpmndefaults = pl[par.helper_dpmn]()
pl.d1defaults = pl[par.helper_d1]()
pl.d2defaults = pl[par.helper_d2]()
pl.actionchannels = pl[par.helper_actionchannels]()
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

In [16]:
#MODIFIERS - init_params.py 

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)
    
def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)
    
def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)
    
def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)
    
def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)
    
def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self): 
    self.actionchannels = par.helper_actionchannels(self.channels)
    
#POPDATA + PATHWAYS - popconstruct_copy.py

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)
    
def codeblock_poppathways(self): 
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways) 

#CONNECTIVITY GRIDS - popconstruct_copy.py

def codeblock_connAMPA(self):
    self.connectivity_AMPA, self.meaneff_AMPA = popconstruct.helper_connectivityAMPA(self.popdata, self.pathways)

def codeblock_connGABA(self): 
    self.connectivity_GABA, self.meaneff_GABA = popconstruct.helper_connectivityGABA(self.popdata, self.pathways) 

def codeblock_connNMDA(self): 
    self.connectivity_NMDA, self.meaneff_NMDA = popconstruct.helper_connectivityNMDA(self.popdata, self.pathways) 

In [17]:
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)
pl.add(codeblock_connAMPA)
pl.add(codeblock_connGABA)
pl.add(codeblock_connNMDA)

In [18]:
#DEFAULT VALUES 

environment = {
    'params': None, 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'newpathways' : None
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [19]:
results.keys()

dict_keys(['params', 'pops', 'receps', 'base', 'dpmns', 'd1', 'd2', 'channels', 'newpathways', 'celldefaults', 'popspecific', 'receptordefaults', 'basestim', 'dpmndefaults', 'd1defaults', 'd2defaults', 'actionchannels', 'popdata', 'pathways', 'connectivity_AMPA', 'meaneff_AMPA', 'connectivity_GABA', 'meaneff_GABA', 'connectivity_NMDA', 'meaneff_NMDA'])

In [20]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [21]:
#CHANGE VALUES - NOT ALL 

environment = {
    'params': pd.DataFrame([[4, 1]], columns=['N', 'C']), 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : pd.DataFrame([[4],[5]], columns=['action']),
    'newpathways' : None
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [22]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [23]:
#CHANGE VALUES - ALL 

environment = {
    'params': pd.DataFrame([[8, 1]], columns=['N', 'C']),
    'pops' : {'FSI': {'Taum': [60]}}, 
    'receps' : pd.DataFrame([[100, 100]], columns=['Tau_AMPA', 'Tau_GABA']), 
    'base' : {'FSI': {'FreqExt_AMPA': [100]}}, 
    'dpmns' : pd.DataFrame([[400, 100]], columns=['dpmn_wmax', 'dpmn_tauDOP']),
    'd1' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'd2' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'channels' : pd.DataFrame([[8],[9]], columns=['action']),
    'newpathways' : pd.DataFrame([['LIP', 'D1STR', 'AMPA', 'syn', 700, 0.027]], columns=['src', 'dest', 'receptor', 'type', 'con', 'eff'])
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [24]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,8`,1`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [25]:
#BACK TO DEFAULT VALUES

environment = {
    'params': None, 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'newpathways' : None
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [26]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`
